In [20]:
#task 1
import numpy as np
bugs=[]
with open("input_day24.txt","r") as file:

    fileLines=file.readlines()
    for j,line in enumerate(fileLines):
        for i,e in enumerate(line[:-1]):
            if e == '#':
                bugs.append((j,i))
                
print(len(bugs))

def getOffsetFromDirection(direction):
    if direction == 1:
        return np.array([-1,0])
    elif direction == 2:
        return np.array([0,1])
    elif direction == 3:
        return np.array([1,0])
    elif direction == 4:
        return np.array([0,-1])
    else:
        return 0

def getNumberOfAdjacentBugs(bugs,j,i):
    num = 0
    #print(bugs,j,i)
    for direction in range(1,5):
        index = np.array([j,i]) + getOffsetFromDirection(direction)
        #print(index)
        if tuple(index) in bugs:
            num += 1
    return num

def visualize(bugs):
    outputStr = ''
    for j in range(0,5):
        for i in range(0,5):
            if (j,i) in bugs:
                outputStr += '#'
            else:
                outputStr += '.'
        outputStr += '\n'
    print(outputStr)

def timestep(bugs):
    #print(bugs)
    newBugs=[]
    for j in range(0,5):
        for i in range(0,5):
            isBug = False
            if (j,i) in bugs:
                isBug = True
            numAdjacentBugs = getNumberOfAdjacentBugs(bugs,j,i)
            if isBug:
                if numAdjacentBugs == 1:
                    newBugs.append((j,i))
            else:
                if numAdjacentBugs == 1 or numAdjacentBugs == 2:
                    newBugs.append((j,i))
    return newBugs
    
def runUntilRepeatedPopulation(bugs):
    bugs.sort()
    history=[bugs]
    t = 1
    while True:
        print("Timestep: ", t)
        newBugs = timestep(history[-1])
        newBugs.sort()
        
        #visualize(newBugs)

        for b in history:
            if b == newBugs:
                print("Found former population!")
                return newBugs
            
        history.append(newBugs)
        t+=1
            
repeatedPopulation=runUntilRepeatedPopulation(bugs)

print(repeatedPopulation)

def getRating(population):
    rating=0
    for b in population:
        index = b[0] * 5 + b[1]
        rating += 2**index
    return rating
print("Biodiversity rating: ", getRating(repeatedPopulation))

11
Timestep:  1
Timestep:  2
Timestep:  3
Timestep:  4
Timestep:  5
Timestep:  6
Timestep:  7
Timestep:  8
Timestep:  9
Timestep:  10
Timestep:  11
Timestep:  12
Timestep:  13
Timestep:  14
Timestep:  15
Timestep:  16
Timestep:  17
Timestep:  18
Timestep:  19
Timestep:  20
Timestep:  21
Timestep:  22
Timestep:  23
Timestep:  24
Timestep:  25
Timestep:  26
Timestep:  27
Timestep:  28
Timestep:  29
Timestep:  30
Timestep:  31
Timestep:  32
Timestep:  33
Timestep:  34
Timestep:  35
Timestep:  36
Found former population!
[(0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)]
Biodiversity rating:  32513278


In [13]:
#task 2
import numpy as np
bugs=[]
with open("input_day24.txt","r") as file:

    fileLines=file.readlines()
    for j,line in enumerate(fileLines):
        for i,e in enumerate(line[:-1]):
            if e == '#':
                bugs.append((j,i))
                

def getOffsetFromDirection(direction):
    if direction == 1:
        return np.array([-1,0])
    elif direction == 2:
        return np.array([0,1])
    elif direction == 3:
        return np.array([1,0])
    elif direction == 4:
        return np.array([0,-1])
    else:
        return 0

def getNumberOfAdjacentBugs(population,level,j,i):
    if j == 2 and i == 2:
        return 0
    num = 0
    #print(population,level,j,i)
    for direction in range(1,5):
        index = np.array([j,i]) + getOffsetFromDirection(direction)
        #print(index)
        # check current level
        if tuple(index) in population[level]:
            num += 1
        # check lower level if on boarder
        if level != min(*population.keys()):
            if index[0] == -1 and (1,2) in population[level-1]:
                num+=1
            if index[0] == 5 and (3,2) in population[level-1]:
                num+=1
            if index[1] == -1 and (2,1) in population[level-1]:
                num+=1
            if index[1] == 5 and (2,3) in population[level-1]:
                num+=1
        # check higher level if close to center
        if level != max(*population.keys()):
            if tuple(index) == (2,2):
                if (j,i) == (1,2):
                    for iRec in range(0,5):
                        if (0,iRec) in population[level+1]:
                            num+=1
                if (j,i) == (3,2):
                    for iRec in range(0,5):
                        if (4,iRec) in population[level+1]:
                            num+=1
                if (j,i) == (2,1):
                    for jRec in range(0,5):
                        if (jRec,0) in population[level+1]:
                            num+=1
                if (j,i) == (2,3):
                    for jRec in range(0,5):
                        if (jRec,4) in population[level+1]:
                            num+=1
                        
    return num

def visualize(population):
    for level in population:
        outputStr = 'Depth '+str(level)+'\n'
        for j in range(0,5):
            for i in range(0,5):
                if (j,i) in population[level]:
                    outputStr += '#'
                elif (j,i) == (2,2):
                    outputStr += '?'
                else:
                    outputStr += '.'
            outputStr += '\n'
        print(outputStr)

def numberOfBugsPerLevel(population):
    numBugs={}
    for level in population:
        numBugs[level]= len(population[level])
    return numBugs

def addEntryToPopulation(population,level,j,i):
    if level in population:
        population[level].append((j,i))
    else:
        population[level] = [(j,i)]

def timestep(population):
    #print(bugs)
    newPopulation={}
    numBugsPerLevel=numberOfBugsPerLevel(population)
    
    # check if new level has to be added
    levelRange=[*numBugsPerLevel.keys()]
    minLevel = min(levelRange)
    maxLevel = max(levelRange)
    if numBugsPerLevel[minLevel] != 0:
        minLevel -= 1
        population[minLevel] = []
    if numBugsPerLevel[maxLevel] != 0:
        maxLevel += 1
        population[maxLevel] = []
    #print(population)
    for level in range(minLevel,maxLevel+1):
        newPopulation[level]=[]
        for j in range(0,5):
            for i in range(0,5):
                if (j,i) == (2,2):
                    continue
                isBug = False
                if (j,i) in population[level]:
                    isBug = True
                numAdjacentBugs = getNumberOfAdjacentBugs(population,level,j,i)
                #print(level,j,i," -> ", isBug, numAdjacentBugs)
                if isBug:
                    if numAdjacentBugs == 1:
                        addEntryToPopulation(newPopulation,level,j,i)
                else:
                    if numAdjacentBugs == 1 or numAdjacentBugs == 2:
                        addEntryToPopulation(newPopulation,level,j,i)
    #print("->",newPopulation)
    return newPopulation
    
def runTimesteps(bugs,timesteps):
    population={0: bugs}
    for t in range(0,timesteps):
        print("Timestep: ", t)
        visualize(population)
        newPopulation = timestep(population)
        population,newPopulation = newPopulation,_
    return population

pop = runTimesteps(bugs,200)
visualize(pop)
numBugPerLevel=numberOfBugsPerLevel(pop)
numBugs = sum(numBugPerLevel[l] for l in numBugPerLevel)
print(numBugs)

Timestep:  0
Depth 0
.#.#.
.##..
.#?..
.###.
##..#

-> {-1: [(1, 2), (2, 1), (2, 3)], 0: [(0, 0), (0, 1), (0, 4), (1, 0), (1, 2), (1, 3), (2, 0), (2, 3), (3, 0), (3, 3), (3, 4), (4, 0), (4, 2), (4, 3)], 1: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)]}
Timestep:  1
Depth -1
.....
..#..
.#?#.
.....
.....

Depth 0
##..#
#.##.
#.?#.
#..##
#.##.

Depth 1
#####
#....
#.?..
#....
#####

-> {-2: [], -1: [(0, 2), (1, 1), (1, 3), (2, 0), (2, 4), (3, 1), (3, 3)], 0: [(3, 1), (4, 1), (4, 2)], 1: [(1, 1), (1, 2), (1, 3), (1, 4), (2, 1), (2, 4), (3, 1), (3, 2), (3, 3), (3, 4)], 2: []}
Timestep:  2
Depth -2
.....
.....
..?..
.....
.....

Depth -1
..#..
.#.#.
#.?.#
.#.#.
.....

Depth 0
.....
.....
..?..
.#...
.##..

Depth 1
.....
.####
.#?.#
.####
.....

Depth 2
.....
.....
..?..
.....
.....

-> {-2: [(1, 2), (2, 1), (2, 3)], -1: [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (4, 1), (4, 3)], 0: [(2, 1), (3, 0), (3, 1), (3, 2), (4, 0), (4, 

-> {-11: [], -10: [(0, 2), (1, 1), (1, 3), (2, 0), (2, 4), (3, 1), (3, 2), (3, 3)], -9: [(0, 1), (0, 3), (1, 0), (1, 1), (1, 3), (1, 4), (3, 0), (3, 4)], -8: [(2, 0), (2, 1), (2, 3), (2, 4), (3, 0), (3, 1), (3, 3), (3, 4), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)], -7: [(0, 0), (0, 2), (0, 4), (1, 0), (1, 1), (1, 3), (1, 4), (2, 0), (2, 1), (2, 3), (2, 4), (3, 1), (3, 3), (4, 0), (4, 1), (4, 3), (4, 4)], -6: [(2, 0), (2, 4), (3, 0), (3, 1), (3, 3), (3, 4), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)], -5: [(3, 1), (3, 3), (4, 0), (4, 4)], -4: [(1, 0), (1, 1), (1, 3), (1, 4), (2, 1), (2, 3), (3, 1), (3, 3), (4, 0), (4, 4)], -3: [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 3), (3, 0), (3, 4), (4, 0), (4, 4)], -2: [(0, 1), (1, 0), (3, 0), (3, 1), (3, 3), (3, 4), (4, 0)], -1: [(0, 0), (0, 2), (0, 3), (1, 4), (3, 0), (3, 3), (3, 4), (4, 0), (4, 2), (4, 4)], 0: [(0, 2), (0, 3), (0, 4), (1, 1), (1, 2), (1, 3), (1, 4), (3, 1), (3, 3), (3, 4), (4, 1), (4, 2), (4, 3), (4, 4)], 1: [(0, 0), (0, 1

-> {-16: [], -15: [(0, 2), (1, 1), (1, 3), (2, 0), (2, 4), (3, 1), (3, 3)], -14: [(0, 1), (0, 3), (1, 0), (1, 1), (1, 3), (1, 4)], -13: [(1, 2), (2, 0), (2, 4), (3, 1), (3, 3), (4, 0), (4, 4)], -12: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (2, 0), (2, 1), (2, 3), (2, 4), (3, 2)], -11: [(1, 2), (2, 1), (2, 3), (3, 2), (4, 1), (4, 3)], -10: [(0, 0), (0, 1), (0, 3), (0, 4), (1, 0), (1, 1), (1, 3), (1, 4), (2, 0), (2, 1), (2, 3), (2, 4), (3, 2)], -9: [(2, 0), (2, 1), (2, 3), (2, 4), (3, 1), (3, 3), (4, 0), (4, 2), (4, 4)], -8: [(1, 0), (1, 1), (1, 3), (1, 4), (4, 0), (4, 4)], -7: [(0, 0), (0, 4), (1, 0), (1, 2), (1, 4), (2, 0), (2, 4)], -6: [(0, 0), (0, 1), (0, 3), (0, 4), (1, 1), (1, 3), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)], -5: [(0, 1), (0, 2), (0, 4), (1, 3), (2, 4), (3, 0), (3, 3), (4, 1), (4, 4)], -4: [(0, 4), (1, 4), (2, 1), (2, 3), (2, 4), (3, 1), (3, 3), (3, 4), (4, 0), (4, 1), (4, 3), (4, 4)], -3: [(0, 0), (1, 0), (1, 1), (1, 3), (1, 4), 


Depth -16
.....
##.##
..?..
.....
.....

Depth -15
.....
.....
..?..
.....
..#..

Depth -14
#...#
#...#
..?..
.....
.....

Depth -13
.....
.....
..?..
#...#
.#.#.

Depth -12
.....
.....
##?##
##.##
.###.

Depth -11
.....
.....
..?..
##.##
.###.

Depth -10
#...#
#...#
#.?.#
##.##
.#.#.

Depth -9
.#.#.
##.##
..?..
.#.#.
#...#

Depth -8
..#..
..#..
..?..
#####
#...#

Depth -7
#.#..
.#.#.
#.?##
.#.#.
##.##

Depth -6
.....
...##
..?#.
.....
..##.

Depth -5
.....
...##
..?#.
.###.
#####

Depth -4
##.#.
##...
##?..
.....
.....

Depth -3
##..#
.#..#
##?..
.#.##
##..#

Depth -2
##..#
....#
##?.#
##...
...##

Depth -1
#.###
#..#.
#.?.#
.##.#
#####

Depth 0
#.###
.#.#.
..?.#
#....
.#.#.

Depth 1
##..#
##...
..?.#
...##
..#..

Depth 2
.###.
###..
#.?..
#....
.##..

Depth 3
...#.
..#.#
.#?..
.#.#.
#.#.#

Depth 4
#..#.
.##.#
##?..
#...#
####.

Depth 5
..#.#
..###
.#?..
##.##
#.#.#

Depth 6
#.###
##.#.
..?..
#....
#....

Depth 7
..##.
###..
#.?#.
####.
...##

Depth 8
..###
....#
..?..
....#
..###

D

-> {-22: [(1, 2), (2, 1), (2, 3)], -21: [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 2), (3, 4), (4, 1), (4, 3)], -20: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (1, 2), (1, 4), (2, 0), (2, 4), (3, 2)], -19: [(0, 2), (1, 0), (1, 1), (1, 3), (1, 4), (3, 2), (4, 1), (4, 3)], -18: [(3, 0), (3, 1), (3, 3), (3, 4)], -17: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (1, 1), (1, 3), (1, 4), (2, 0), (2, 4), (3, 0), (3, 1), (3, 3), (3, 4), (4, 0), (4, 2), (4, 4)], -16: [(3, 1), (3, 2), (3, 3), (4, 1), (4, 3)], -15: [(0, 0), (0, 1), (0, 3), (0, 4), (1, 1), (1, 3), (2, 0), (2, 4), (3, 0), (3, 4), (4, 0), (4, 1), (4, 3), (4, 4)], -14: [(0, 0), (0, 1), (0, 3), (0, 4), (1, 0), (1, 4), (2, 0), (2, 4), (3, 1), (3, 3), (4, 2)], -13: [(0, 2), (1, 1), (1, 3), (2, 0), (2, 4), (3, 0), (3, 4), (4, 1), (4, 3)], -12: [(0, 2), (1, 0), (1, 1), (1, 3), (1, 4), (3, 1), (3, 3), (4, 0), (4, 2), (4, 4)], -11: [(4, 0), (4, 1), (4, 2), (4, 3), (4, 4)], -10: [(0, 2), (2, 0), (2, 4), (3, 0), (3, 1), (3, 3), (3

...

Depth -6
.##.#
#....
#.?..
.#.##
.###.

Depth -5
.#.#.
##.#.
#.?..
#..#.
##..#

Depth -4
...##
...#.
..?.#
...#.
..#.#

Depth -3
#.#..
##...
..?..
##.#.
.....

Depth -2
..#.#
...#.
#.?.#
.#...
#.#..

Depth -1
##.#.
.#..#
#.?##
#....
.##.#

Depth 0
####.
.....
..?..
#..##
.###.

Depth 1
.....
...#.
..?..
#####
#....

Depth 2
.#.#.
#####
.#?..
.#.##
##..#

Depth 3
.#.##
#..##
..?.#
##...
#....

Depth 4
#####
#....
#.?..
.....
....#

Depth 5
.##..
###..
#.?##
..###
..##.

Depth 6
.....
.....
..?..
##.##
#####

Depth 7
#...#
....#
..?..
...##
#...#

Depth 8
#####
##.#.
..?.#
.....
.....

Depth 9
.....
...#.
..?#.
.#.#.
.#.#.

Depth 10
#..##
##...
#.?##
..##.
..#..

Depth 11
.##.#
#..##
#.?.#
.##.#
##..#

Depth 12
.....
.....
..?.#
###.#
#....

Depth 13
.##.#
#..#.
..?..
#.#..
...#.

Depth 14
..#.#
...##
..?.#
###.#
#.#..

Depth 15
#...#
....#
#.?.#
##..#
##.#.

Depth 16
.....
.....
..?.#
....#
..##.

Depth 17
.##.#
##.##
#.?.#
#..#.
#.###

Depth 18
##..#
##...
##?..
...#.
..#.#

Depth 19
.

Depth 36
.###.
###..
..?..
###..
.###.

Depth 37
..###
...##
..?..
...##
..###

Depth 38
#.#.#
##.#.
..?..
##.#.
#.#.#

Depth 39
#####
....#
..?..
....#
#####

Depth 40
.##..
##...
.#?..
##...
.##..

Depth 41
####.
.#.#.
#.?..
.#.#.
####.

Depth 42
#....
.#..#
#.?..
.#..#
#....

Depth 43
.####
#..#.
..?.#
#..#.
.####

Depth 44
.##.#
#.##.
..?..
#.##.
.##.#

Depth 45
####.
.#...
#.?..
.#...
####.

Depth 46
####.
...#.
..?#.
...#.
####.

Depth 47
#.##.
##...
..?#.
##...
#.##.

Depth 48
##..#
##.#.
#.?.#
##.#.
##..#

Depth 49
.#.#.
#..##
..?.#
#..##
.#.#.

Depth 50
#.##.
.#.#.
..?..
.#.#.
#.##.

Depth 51
#.###
##.#.
..?..
##.#.
#.###

Depth 52
.##..
.#...
#.?..
.#...
.##..

Depth 53
.###.
.#.#.
#.?..
.#.#.
.###.

Depth 54
#####
#.##.
##?##
#.##.
#####

Depth 55
.....
.....
.#?#.
.....
.....

Depth 56
#..#.
.#..#
#.?..
.#..#
#..#.

Depth 57
###..
###..
..?#.
###..
###..

Depth 58
##.##
.#.##
..?##
.#.##
##.##

Depth 59
#.##.
##...
..?#.
##...
#.##.

Depth 60
#...#
...##
..?..
...##
#...#



-> {-99: [(1, 2), (2, 1), (2, 3)], -98: [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 1), (3, 3), (3, 4), (4, 1), (4, 2), (4, 3)], -97: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (1, 2), (1, 4), (2, 0), (2, 4), (3, 1), (3, 3), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)], -96: [(1, 0), (1, 1), (1, 3), (1, 4)], -95: [(4, 2)], -94: [(0, 0), (0, 4), (1, 0), (1, 4)], -93: [(3, 0), (3, 4), (4, 1), (4, 3)], -92: [(2, 0), (2, 1), (2, 3), (2, 4), (3, 0), (3, 1), (3, 3), (3, 4), (4, 1), (4, 2), (4, 3)], -91: [(3, 0), (3, 1), (3, 3), (3, 4), (4, 1), (4, 2), (4, 3)], -90: [(0, 0), (0, 4), (1, 0), (1, 4), (2, 0), (2, 4), (3, 0), (3, 1), (3, 3), (3, 4), (4, 1), (4, 3)], -89: [(0, 1), (0, 3), (1, 0), (1, 1), (1, 3), (1, 4), (3, 1), (3, 3), (4, 0), (4, 4)], -88: [(0, 2), (1, 2), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (4, 0), (4, 4)], -87: [(0, 2), (1, 1), (1, 3), (2, 0), (2, 4), (4, 0), (4, 1), (4, 3), (4, 4)], -86: [(0, 1), (0, 3), (1, 1), (1, 2), (1, 3), (3, 1), (3, 3)], -85: [(0, 0), (0, 1), (0, 


Depth -17
##...
.##..
##?##
#.###
.#.##

Depth -16
##.##
#....
..?.#
#..##
####.

Depth -15
.....
.....
#.?##
...#.
...##

Depth -14
.#...
....#
#.?.#
.#..#
##..#

Depth -13
..###
.#..#
#.?.#
.#..#
..###

Depth -12
####.
.#..#
#.?.#
#....
..#..

Depth -11
#.#.#
.#.##
#.?..
....#
#...#

Depth -10
####.
##..#
#.?##
.....
#.#..

Depth -9
#.###
.##.#
##?.#
.#.##
##.#.

Depth -8
#.#.#
##.##
#.?..
#..#.
...#.

Depth -7
####.
##.##
#.?#.
#.###
#...#

Depth -6
....#
....#
#.?#.
#.#..
.....

Depth -5
###.#
#..##
..?..
....#
##.#.

Depth -4
.....
.#...
..?..
##..#
.###.

Depth -3
.####
#..#.
#.?.#
...#.
..#.#

Depth -2
##.#.
#...#
..?..
....#
#####

Depth -1
...#.
.#.##
#.?##
...#.
#..#.

Depth 0
..#..
.....
..?##
##...
#.#..

Depth 1
#.###
##.##
#.?..
#....
.....

Depth 2
#...#
....#
..?..
#####
###.#

Depth 3
##.##
##.##
..?.#
#.###
..#.#

Depth 4
.....
.###.
.#?#.
##.#.
#.###

Depth 5
.....
..#..
.#?#.
##.#.
###.#

Depth 6
..#..
#....
.#?.#
#....
.....

Depth 7
.##.#
.#...
#.?..
.#.##
....#


-> {-101: [], -100: [(0, 2), (1, 1), (1, 3), (2, 0), (2, 4), (3, 1), (3, 2), (3, 3)], -99: [(0, 1), (0, 3), (1, 0), (1, 1), (1, 3), (1, 4), (3, 0), (3, 4)], -98: [(2, 0), (2, 1), (2, 3), (2, 4), (3, 0), (3, 1), (3, 3), (3, 4), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)], -97: [(0, 0), (0, 2), (0, 4), (1, 0), (1, 1), (1, 3), (1, 4), (2, 0), (2, 1), (2, 3), (2, 4), (3, 1), (3, 2), (3, 3), (4, 0), (4, 1), (4, 3), (4, 4)], -96: [(2, 0), (2, 4), (3, 0), (3, 1), (3, 3), (3, 4), (4, 0), (4, 2), (4, 4)], -95: [(3, 0), (3, 1), (3, 3), (3, 4), (4, 0), (4, 2), (4, 4)], -94: [(0, 2), (1, 1), (1, 3), (2, 0), (2, 4)], -93: [(0, 2), (1, 1), (1, 3), (2, 0), (2, 4), (3, 0), (3, 4), (4, 1), (4, 3)], -92: [(0, 0), (0, 2), (0, 4), (1, 1), (1, 3), (2, 0), (2, 4), (3, 1), (3, 3), (4, 1), (4, 2), (4, 3)], -91: [(0, 0), (0, 4), (1, 1), (1, 3), (2, 0), (2, 4), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (4, 0), (4, 4)], -90: [(0, 0), (0, 4), (1, 1), (1, 3), (2, 0), (2, 4), (3, 1), (3, 3), (4, 0), (4, 1), (4, 3), (4, 4